In [ ]:
import os
STEPS = "./steps"
if not os.path.isdir(STEPS):
    os.mkdir(STEPS)

### Build and Run Dataset Downloader

In [9]:
!cd sub/paralells ; cargo build --release ; ./target/release/paralells --inputfolder input --outputfile result.csv

 --> src/main.rs:2:5
  |
2 | use csv::StringRecord;
  |     ^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:3:5
  |
3 | use http::Request;
  |     ^^^^^^^^^^^^^

  --> src/main.rs:9:16
   |
 9 |     fs::{self, DirEntry, File, FileType},
   |                ^^^^^^^^        ^^^^^^^^
10 |     io::{Cursor, Read, Write},
11 |     path::{self, Path, PathBuf},
   |            ^^^^

   --> src/main.rs:121:37
    |
121 |             InputFile::Real { path, baseid } => Some(vec![path.clone()]),
    |                                     ^^^^^^ help: try ignoring the field: `baseid: _`
    |
    = note: `#[warn(unused_variables)]` on by default

    Finished `release` profile [optimized] target(s) in 0.18s


In [ ]:
from ultralytics import YOLO
from PIL import Image

# 1. Load a pre-trained segmentation model
# The 'yolov8n-seg.pt' is a small and fast segmentation model.
# Other options include yolov8s-seg.pt, yolov8m-seg.pt, etc.
model = YOLO('yolo11x-seg.pt')
source_image = 'https://ultralytics.com/images/bus.jpg'
results = model.predict(source="image.jpg", device='cuda', save=True)

# 4. Process results (optional)
# The 'results' object contains all the information.
# For segmentation, you can access the masks.
if results[0].masks:
    print(f"Found {len(results[0].masks)} objects with masks.")
    # You can access mask data via results[0].masks.data
else:
    print("No segmentation masks were found.")

# The predicted image with masks will be saved in a 'runs/segment/predict' directory.
print("\nSegmentation complete! Check the 'runs/segment/predict' folder for the output image.")


image 1/1 c:\FastProjects\DeepLearningRepo\image.jpg: 640x480 1 apple, 1 dining table, 36.9ms
Speed: 3.1ms preprocess, 36.9ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\FastProjects\DeepLearningRepo\runs\segment\predict
Found 2 objects with masks.

Segmentation complete! Check the 'runs/segment/predict' folder for the output image.


In [12]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
INPUT_FOLDER = "./sub/paralells/steps/convertedtemp/" 

# 2. Set the path where the processed images will be saved.
OUTPUT_FOLDER = "processed_images" 

# 3. Define the target object classes the script should look for.
TARGET_CLASSES = ['banana', 'apple', 'avocado'] 

# 4. Define the final output size for the images.
FINAL_SIZE = (720, 720) 

# 5. Choose the model. 'yolov8n.pt' is small and fast, perfect for this.
MODEL_NAME = 'yolo11x-seg.pt' 


def preprocess_images():
    """
    Detects target objects, crops a 1:1 square around them, 
    resizes to a final size, and saves the results.
    """
    # Load the pre-trained YOLO model
    # The model will be downloaded automatically on the first run.
    print(f"Loading YOLO model: {MODEL_NAME}...")
    try:
        model = YOLO(MODEL_NAME)
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Please ensure you have an internet connection for the first run.")
        return

    # Create the output directory if it doesn't exist
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"Output will be saved to: {OUTPUT_FOLDER}")

    # List all files in the input folder
    if not os.path.exists(INPUT_FOLDER):
        print(f"Error: Input folder not found at '{INPUT_FOLDER}'")
        return
        
    image_files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print(f"No images found in '{INPUT_FOLDER}'. Please check the path.")
        return

    print(f"Found {len(image_files)} images to process.")
    processed_count = 0

    # Process each image file
    for filename in image_files:
        image_path = os.path.join(INPUT_FOLDER, filename)
        
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"--- Skipping {filename}: Could not read image file.")
            continue
        
        img_h, img_w = image.shape[:2]
        
        # Run YOLO detection
        results = model(image, verbose=False) # Set verbose to False to reduce console output
        
        best_detection = None
        highest_confidence = 0.0

        # Find the best detection among the target classes
        for result in results:
            if result.boxes is None:
                continue
            for box in result.boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                confidence = float(box.conf[0])
                
                if class_name in TARGET_CLASSES and confidence > highest_confidence:
                    highest_confidence = confidence
                    best_detection = box.xyxy[0].cpu().numpy() # [x1, y1, x2, y2]
        
        if best_detection is not None:
            # Calculate the center point of the detected object
            x1, y1, x2, y2 = best_detection
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # Calculate the largest possible square crop
            # The half-size is the minimum distance from the center to any edge
            dist_to_top = center_y
            dist_to_bottom = img_h - center_y
            dist_to_left = center_x
            dist_to_right = img_w - center_x
            
            half_size = int(min(dist_to_top, dist_to_bottom, dist_to_left, dist_to_right))

            # Define the crop box coordinates
            crop_x1 = center_x - half_size
            crop_y1 = center_y - half_size
            crop_x2 = center_x + half_size
            crop_y2 = center_y + half_size
            
            # Crop the image
            cropped_image = image[crop_y1:crop_y2, crop_x1:crop_x2]
            
            # Resize the cropped image to the final target size
            resized_image = cv2.resize(cropped_image, FINAL_SIZE, interpolation=cv2.INTER_AREA)
            
            # Save the processed image
            output_path = os.path.join(OUTPUT_FOLDER, filename)
            cv2.imwrite(output_path, resized_image)
            print(f"-> Processed '{filename}' and saved to output folder.")
            processed_count += 1
        else:
            print(f"--- Skipping '{filename}': No target object found.")

    print(f"\nProcessing complete. Successfully processed {processed_count} out of {len(image_files)} images.")

if __name__ == "__main__":
    preprocess_images()


Loading YOLO model: yolo11x-seg.pt...
Output will be saved to: processed_images
Found 420 images to process.
-> Processed 'd4e02b45aa8418451e61c741ead3aaf1.png' and saved to output folder.
-> Processed '1abfee67ce5070983b76f22f4f246809.png' and saved to output folder.
-> Processed '1918e146138d7bb7ced75d3c2019d31a.png' and saved to output folder.
-> Processed 'ba83b2b72dcb404ccdcb52adeb5e0923.png' and saved to output folder.
-> Processed '81486beccfba6145f7d585e85e6c42fe.png' and saved to output folder.
-> Processed '15e9b0ef17ae9697a4f822c5d8c5225c.png' and saved to output folder.
-> Processed '2df46740b75cca8829da1a3756d98aa9.png' and saved to output folder.
-> Processed '64f3c6849e6e072885aac4bbfbd7bd53.png' and saved to output folder.
-> Processed 'e9357e97e3c4b0d86fd026b4052b1745.png' and saved to output folder.
-> Processed '624f6b3d67001b33be04036766f0d5d6.png' and saved to output folder.
-> Processed '2df79d18c797040d7cb3bff3a3c7b472.png' and saved to output folder.
-> Processe

Based on the previous step, find how many images succeeded in processing, and create a new CSV with only those only.

In [13]:
CSV_FILE = "./sub/paralells/result.csv"
PROCESSED_FOLDER = "./processed_images/"

import os
import csv

def filter_csv_by_processed_images(input_csv, processed_folder, output_csv):
    """
    Reads input_csv, keeps only rows where the image (first column) exists in processed_folder,
    and writes the filtered rows to output_csv.
    """
    processed_images = set(os.listdir(processed_folder))
    filtered_rows = []

    with open(input_csv, newline='') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        filtered_rows.append(header)
        for row in reader:
            if row and row[0] in processed_images:
                filtered_rows.append(row)

    with open(output_csv, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(filtered_rows)
    print(f"Filtered CSV saved to {output_csv} with {len(filtered_rows)-1} entries.")

filter_csv_by_processed_images("./sub/paralells/result.csv", "./processed_images/", "./filtered_result.csv")

Filtered CSV saved to ./filtered_result.csv with 373 entries.
